In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import pickle
import numpy as np
from sklearn.metrics import accuracy_score
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

C:\Users\master\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
df = pd.read_csv('../submission files/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
print(df.info())
df['Age'] = df['Age'].fillna(df['Age'].mean())
print(df.info())
labels = ['child','adult', 'old']
df['age_group'] = pd.cut(df['Age'], bins=3, labels=labels)
df['age_group']= df['age_group'].fillna('adult')
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          882 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Sur

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
age_group        0
dtype: int64

In [4]:
labels = ['cheap', 'normal', 'expensive']
df['Fare_group'] = pd.cut(df['Fare'], bins=3, labels=labels)
df['Fare_group']= df['Fare_group'].fillna('cheap')
print(df.isnull().sum())
df

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
age_group        0
Fare_group       0
dtype: int64


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age_group,Fare_group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,child,cheap
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,adult,cheap
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,child,cheap
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,adult,cheap
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,adult,cheap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,adult,cheap
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,child,cheap
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,7.0,1,2,W./C. 6607,23.4500,NaN,S,child,cheap
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,child,cheap


In [5]:
df.drop(columns= ['PassengerId', 'Name', 'Cabin', 'Embarked', 'Ticket','Age', 'Fare'], inplace=True)
df

,Survived,Pclass,Sex,SibSp,Parch,age_group,Fare_group
0,0,3,male,1,0,child,cheap
1,1,1,female,1,0,adult,cheap
2,1,3,female,0,0,child,cheap
3,1,1,female,1,0,adult,cheap
4,0,3,male,0,0,adult,cheap
...,...,...,...,...,...,...,...
886,0,2,male,0,0,adult,cheap
887,1,1,female,0,0,child,cheap
888,0,3,female,1,2,child,cheap
889,1,1,male,0,0,child,cheap


In [6]:
y = df.iloc[:, 0].values
X = df.iloc[:,1:].values
X

array([[3, 'male', 1, 0, 'child', 'cheap'],
       [1, 'female', 1, 0, 'adult', 'cheap'],
       [3, 'female', 0, 0, 'child', 'cheap'],
       ...,
       [3, 'female', 1, 2, 'child', 'cheap'],
       [1, 'male', 0, 0, 'child', 'cheap'],
       [3, 'male', 0, 0, 'adult', 'cheap']], dtype=object)

In [7]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,4,-1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[0.0 1.0 0.0 ... 3 1 0]
 [1.0 0.0 1.0 ... 1 1 0]
 [1.0 0.0 0.0 ... 3 0 0]
 ...
 [1.0 0.0 0.0 ... 3 1 2]
 [0.0 1.0 0.0 ... 1 0 0]
 [0.0 1.0 1.0 ... 3 0 0]]


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)
print(X_train)
print(X_test)
print(y_train)
print(y_test)

[[0.0 1.0 0.0 ... 3 2 0]
 [0.0 1.0 1.0 ... 1 0 0]
 [0.0 1.0 1.0 ... 3 0 0]
 ...
 [0.0 1.0 0.0 ... 3 0 0]
 [0.0 1.0 0.0 ... 3 0 0]
 [0.0 1.0 0.0 ... 3 0 0]]
[[1.0 0.0 1.0 ... 3 1 0]
 [0.0 1.0 1.0 ... 3 0 0]
 [0.0 1.0 0.0 ... 3 0 0]
 ...
 [0.0 1.0 0.0 ... 3 0 0]
 [0.0 1.0 1.0 ... 1 0 0]
 [0.0 1.0 0.0 ... 1 1 0]]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 0 1 1 0 0 0 0 1 1 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 1 0 0 1 0 1 1 0
 1 0 1 0 1 0 1 1 0 0 1 1 0 1 0 1 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0
 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 0 0 1 0 1 1 0
 1 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 1 0 0 1 1 1 0 1 1 1 1 0 0 0 0 0
 1 1 0 1 1 1 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0
 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1
 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0
 0 1 0 0 1 1 

In [9]:
sc = StandardScaler()
X_train[:, 2:] = sc.fit_transform(X_train[:, 2:])
X_test[:, 2:] = sc.transform(X_test[:, 2:])
print(X_train)
print(X_test)

[[0.0 1.0 -0.9805266477189967 ... 0.8249189043748203 1.291688566808702
  -0.4811217256593604]
 [0.0 1.0 1.019860094905432 ... -1.557808316375167 -0.4684688677708115
  -0.4811217256593604]
 [0.0 1.0 1.019860094905432 ... 0.8249189043748203 -0.4684688677708115
  -0.4811217256593604]
 ...
 [0.0 1.0 -0.9805266477189967 ... 0.8249189043748203 -0.4684688677708115
  -0.4811217256593604]
 [0.0 1.0 -0.9805266477189967 ... 0.8249189043748203 -0.4684688677708115
  -0.4811217256593604]
 [0.0 1.0 -0.9805266477189967 ... 0.8249189043748203 -0.4684688677708115
  -0.4811217256593604]]
[[1.0 0.0 1.019860094905432 ... 0.8249189043748203 0.41160984951894514
  -0.4811217256593604]
 [0.0 1.0 1.019860094905432 ... 0.8249189043748203 -0.4684688677708115
  -0.4811217256593604]
 [0.0 1.0 -0.9805266477189967 ... 0.8249189043748203 -0.4684688677708115
  -0.4811217256593604]
 ...
 [0.0 1.0 -0.9805266477189967 ... 0.8249189043748203 -0.4684688677708115
  -0.4811217256593604]
 [0.0 1.0 1.019860094905432 ... -1.5578

In [44]:
model = Sequential()
model.add(Dense(11, init='uniform', activation='relu', input_dim=11))
model.add(BatchNormalization())
model.add(Dense(64, init = 'uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, init = 'uniform', activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(64, init = 'uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, init = 'uniform', activation='relu'))
model.add(Dense(16, init = 'uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
model.compile(loss='binary_crossentropy',
 optimizer='adam',
 metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 11)                132       
_________________________________________________________________
batch_normalization_9 (Batch (None, 11)                44        
_________________________________________________________________
dense_27 (Dense)             (None, 64)                768       
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_13 (Dropout)         (None, 32)                0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 32)               

C:\Users\master\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(11, activation="relu", input_dim=11, kernel_initializer="uniform")`
  
C:\Users\master\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\master\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="uniform")`
  
C:\Users\master\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_initializer="uniform")`
  if __name__ == '__main__':
C:\Users\master\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", k

In [45]:
model.fit(X_train, y_train,validation_split=0.3,batch_size=32, epochs =100)

Train on 498 samples, validate on 214 samples
Epoch 1/100
498/498 [==============================] - 1s 1ms/step - loss: 0.6925 - accuracy: 0.5683 - val_loss: 0.6918 - val_accuracy: 0.6075
Epoch 2/100
498/498 [==============================] - 0s 64us/step - loss: 0.6896 - accuracy: 0.6064 - val_loss: 0.6895 - val_accuracy: 0.6075
Epoch 3/100
498/498 [==============================] - 0s 90us/step - loss: 0.6675 - accuracy: 0.7390 - val_loss: 0.6829 - val_accuracy: 0.6075
Epoch 4/100
498/498 [==============================] - 0s 73us/step - loss: 0.5635 - accuracy: 0.7952 - val_loss: 0.6561 - val_accuracy: 0.6075
Epoch 5/100
498/498 [==============================] - 0s 62us/step - loss: 0.4852 - accuracy: 0.7952 - val_loss: 0.6340 - val_accuracy: 0.6215
Epoch 6/100
498/498 [==============================] - 0s 76us/step - loss: 0.4479 - accuracy: 0.7992 - val_loss: 0.6375 - val_accuracy: 0.6215
Epoch 7/100
498/498 [==============================] - 0s 64us/step - loss: 0.4553 - accura

498/498 [==============================] - 0s 62us/step - loss: 0.3905 - accuracy: 0.8173 - val_loss: 0.5243 - val_accuracy: 0.7570
Epoch 58/100
498/498 [==============================] - 0s 54us/step - loss: 0.3829 - accuracy: 0.8313 - val_loss: 0.5197 - val_accuracy: 0.7570
Epoch 59/100
498/498 [==============================] - 0s 64us/step - loss: 0.3808 - accuracy: 0.8474 - val_loss: 0.5213 - val_accuracy: 0.7664
Epoch 60/100
498/498 [==============================] - 0s 52us/step - loss: 0.3853 - accuracy: 0.8213 - val_loss: 0.5193 - val_accuracy: 0.7617
Epoch 61/100
498/498 [==============================] - 0s 64us/step - loss: 0.3798 - accuracy: 0.8434 - val_loss: 0.5204 - val_accuracy: 0.7617
Epoch 62/100
498/498 [==============================] - 0s 66us/step - loss: 0.3795 - accuracy: 0.8193 - val_loss: 0.5252 - val_accuracy: 0.7617
Epoch 63/100
498/498 [==============================] - 0s 52us/step - loss: 0.3916 - accuracy: 0.8333 - val_loss: 0.5160 - val_accuracy: 0.761

In [46]:
predictions= model.predict(X_test).tolist()
print(predictions)


[[0.7534602880477905], [0.19216901063919067], [0.14531074464321136], [0.05339082330465317], [0.15636852383613586], [0.05339082330465317], [0.14531074464321136], [0.5792263150215149], [0.09553523361682892], [0.9599935412406921], [0.05339082330465317], [0.23241853713989258], [0.14531074464321136], [0.14531074464321136], [0.04479841887950897], [0.14531074464321136], [0.23241853713989258], [0.060103900730609894], [0.45990100502967834], [0.6162369251251221], [0.12180211395025253], [0.43682029843330383], [0.14531074464321136], [0.14531074464321136], [0.0841446965932846], [0.060103900730609894], [0.12180211395025253], [0.019260192289948463], [0.060103900730609894], [0.45990100502967834], [0.9986323714256287], [0.9665386080741882], [0.06965197622776031], [0.5666226744651794], [0.45990100502967834], [0.060103900730609894], [0.04413287341594696], [0.5598713755607605], [0.06284042447805405], [0.19216901063919067], [0.060103900730609894], [0.1853913962841034], [0.9665386080741882], [0.671405911445

In [47]:
se = pd.Series(predictions)
df['check'] = se
df['check'] = df['check'].str.get(0)

In [48]:
finalpred = []
count0 = 0
count1 = 0
for val in df.check:
    if val >=0.5:
        finalpred.append(1)
        count1 += 1
    else:
        finalpred.append(0)
        count0 += 1

df['Prediction'] = finalpred
print("Count 0:", count0)
print("Count 1:", count1)

Count 0: 839
Count 1: 52


In [39]:
df

,Survived,Pclass,Sex,SibSp,Parch,age_group,Fare_group,check,Prediction
0,0,3,male,1,0,child,cheap,0.735229,1
1,1,1,female,1,0,adult,cheap,0.129783,0
2,1,3,female,0,0,child,cheap,0.129664,0
3,1,1,female,1,0,adult,cheap,0.073232,0
4,0,3,male,0,0,adult,cheap,0.025083,0
...,...,...,...,...,...,...,...,...,...
886,0,2,male,0,0,adult,cheap,NaN,0
887,1,1,female,0,0,child,cheap,NaN,0
888,0,3,female,1,2,child,cheap,NaN,0
889,1,1,male,0,0,child,cheap,NaN,0


In [49]:
filename='titanic8250.sav'
pickle.dump(model, open(filename, 'wb'))